# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,0.28,97,100,3.31,CL,1689889133
1,1,waitangi,-43.9535,-176.5597,7.68,94,84,2.24,NZ,1689889127
2,2,waddan,29.1614,16.1390,29.66,36,0,4.57,LY,1689889542
3,3,hermanus,-34.4187,19.2345,9.33,79,0,3.93,ZA,1689888953
4,4,boras,57.7210,12.9401,13.24,77,82,5.16,SE,1689889543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
cities_map = city_data_df.hvplot.points('Lng','Lat', geo=True, tiles="EsriImagery", frame_height = 600, frame_width = 800, color="City", size="Humidity", alpha=0.78)
# print(city_data_df["Humidity"])
# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria 
        # A max temperature lower than 25 degrees but higher than 16
        # Wind speed less than 4.5 m/s
        # Zero cloudiness

# and drop any results with null values 
perfect_weather = city_data_df.loc[(city_data_df["Max Temp"]<25) & 
                                (city_data_df["Max Temp"]>16) & 
                                (city_data_df["Wind Speed"]<4.5) & 
                                (city_data_df["Cloudiness"]==0)]
# Drop any rows with null values # It doesn't have null values :O
# Display sample data
perfect_weather.dropna(inplace=True)
perfect_weather

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12360\3651117418.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfect_weather.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,129,belo sur tsiribihina,-19.7000,44.5500,19.44,81,0,1.86,MG,1689889614
218,218,saint-pierre,-21.3393,55.4781,21.98,73,0,3.09,RE,1689889664
263,263,mugia,43.1041,-9.2179,18.84,85,0,1.31,ES,1689889689
293,293,colonia,50.9333,6.9500,18.31,76,0,3.60,DE,1689889596
311,311,zhangye,38.9342,100.4517,19.96,35,0,2.68,CN,1689889722
347,347,hebron,31.5333,35.1000,24.97,26,0,1.52,PS,1689889740
391,391,prince rupert,54.3161,-130.3201,17.05,89,0,2.06,CA,1689889766
402,402,minyat an nasr,31.1229,31.6399,24.88,70,0,1.93,EG,1689889772
409,409,esfarayen,37.0765,57.5101,21.81,40,0,2.15,IR,1689889778
524,524,edmundston,47.3737,-68.3251,24.86,48,0,2.06,CA,1689889846


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
129,belo sur tsiribihina,MG,-19.7000,44.5500,81,
218,saint-pierre,RE,-21.3393,55.4781,73,
263,mugia,ES,43.1041,-9.2179,85,
293,colonia,DE,50.9333,6.9500,76,
311,zhangye,CN,38.9342,100.4517,35,
347,hebron,PS,31.5333,35.1000,26,
391,prince rupert,CA,54.3161,-130.3201,89,
402,minyat an nasr,EG,31.1229,31.6399,70,
409,esfarayen,IR,37.0765,57.5101,40,
524,edmundston,CA,47.3737,-68.3251,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "filter":"",
    "bias":"",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    lat=hotel_df.loc[index,"Lat"]
    lon=hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
 
    # Convert the API response to JSON format
    name_address = name_address.json()
 
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
belo sur tsiribihina - nearest hotel: Karibo
saint-pierre - nearest hotel: Tropic Hotel
mugia - nearest hotel: Solpor do Coído
colonia - nearest hotel: Wasserturm Hotel Cologne
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
hebron - nearest hotel: Hebron Hotel
prince rupert - nearest hotel: Crest Hotel
minyat an nasr - nearest hotel: No hotel found
esfarayen - nearest hotel: هتل ساریگل
edmundston - nearest hotel: Station 127
jinchang - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
129,belo sur tsiribihina,MG,-19.7000,44.5500,81,Karibo
218,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
263,mugia,ES,43.1041,-9.2179,85,Solpor do Coído
293,colonia,DE,50.9333,6.9500,76,Wasserturm Hotel Cologne
311,zhangye,CN,38.9342,100.4517,35,Zhangye Hotel 张掖饭店
347,hebron,PS,31.5333,35.1000,26,Hebron Hotel
391,prince rupert,CA,54.3161,-130.3201,89,Crest Hotel
402,minyat an nasr,EG,31.1229,31.6399,70,No hotel found
409,esfarayen,IR,37.0765,57.5101,40,هتل ساریگل
524,edmundston,CA,47.3737,-68.3251,48,Station 127


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
hotels_map = hotel_df.hvplot.points('Lng','Lat', geo=True, tiles="EsriImagery", frame_height = 600, frame_width = 800, color="City", size="Humidity", alpha=0.9, hover_cols=['Hotel Name'])


# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)